<a href="https://colab.research.google.com/github/AyusheeMittal/legendary-guide/blob/master/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 1.x

import os
import cv2
import json
import argparse
import warnings
import numpy as np
import pandas as pd
import keras.backend as K
import matplotlib.pyplot as plt


from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout, Flatten, Dense
from keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

import tensorflow as tf
import tensorflow.contrib.eager as tfe

#tf.enable_eager_execution()

Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(8,2,figsize=(15,40))
    # summarize history for accuracy
    axs[0,0].plot(range(1,len(model_history.history['gender_output_acc'])+1),model_history.history['gender_output_acc'])
    axs[0,0].plot(range(1,len(model_history.history['val_gender_output_acc'])+1),model_history.history['val_gender_output_acc'])
    axs[0,0].set_title('Model gender_output_acc')
    axs[0,0].set_ylabel('gender_output_acc')
    axs[0,0].set_xlabel('Epoch')
    axs[0,0].set_xticks(np.arange(1,len(model_history.history['gender_output_acc'])+1),len(model_history.history['gender_output_acc'])/10)
    axs[0,0].legend(['train', 'val'], loc='best')

    axs[0,1].plot(range(1,len(model_history.history['image_quality_output_acc'])+1),model_history.history['image_quality_output_acc'])
    axs[0,1].plot(range(1,len(model_history.history['val_image_quality_output_acc'])+1),model_history.history['val_image_quality_output_acc'])
    axs[0,1].set_title('Model image_quality_output_acc')
    axs[0,1].set_ylabel('image_quality_output_acc')
    axs[0,1].set_xlabel('Epoch')
    axs[0,1].set_xticks(np.arange(1,len(model_history.history['image_quality_output_acc'])+1),len(model_history.history['image_quality_output_acc'])/10)
    axs[0,1].legend(['train', 'val'], loc='best')

    axs[1,0].plot(range(1,len(model_history.history['pose_output_loss'])+1),model_history.history['pose_output_loss'])
    axs[1,0].plot(range(1,len(model_history.history['val_pose_output_loss'])+1),model_history.history['val_pose_output_loss'])
    axs[1,0].set_title('Model Loss')
    axs[1,0].set_ylabel('Loss')
    axs[1,0].set_xlabel('Epoch')
    axs[1,0].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[1,0].legend(['train', 'val'], loc='best')

    axs[1,1].plot(range(1,len(model_history.history['emotion_output_loss'])+1),model_history.history['emotion_output_loss'])   
    axs[1,1].plot(range(1,len(model_history.history['val_emotion_output_loss'])+1),model_history.history['val_emotion_output_loss'])
    axs[1,1].set_title('Model Loss')
    axs[1,1].set_ylabel('Loss')
    axs[1,1].set_xlabel('Epoch')
    axs[1,1].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[1,1].legend(['train', 'val'], loc='best')

    axs[2,0].plot(range(1,len(model_history.history['age_output_acc'])+1),model_history.history['age_output_acc'])
    axs[2,0].plot(range(1,len(model_history.history['val_age_output_acc'])+1),model_history.history['val_age_output_acc'])
    axs[2,0].set_title('Model age_output_acc')
    axs[2,0].set_ylabel('age_output_acc')
    axs[2,0].set_xlabel('Epoch')
    axs[2,0].set_xticks(np.arange(1,len(model_history.history['age_output_acc'])+1),len(model_history.history['age_output_acc'])/10)
    axs[2,0].legend(['train', 'val'], loc='best')

    axs[2,1].plot(range(1,len(model_history.history['weight_output_acc'])+1),model_history.history['weight_output_acc'])
    axs[2,1].plot(range(1,len(model_history.history['val_weight_output_acc'])+1),model_history.history['val_weight_output_acc'])
    axs[2,1].set_title('Model weight_output_acc')
    axs[2,1].set_ylabel('weight_output_acc')
    axs[2,1].set_xlabel('Epoch')
    axs[2,1].set_xticks(np.arange(1,len(model_history.history['weight_output_acc'])+1),len(model_history.history['weight_output_acc'])/10)
    axs[2,1].legend(['train', 'val'], loc='best')

    axs[3,0].plot(range(1,len(model_history.history['bag_output_acc'])+1),model_history.history['bag_output_acc'])
    axs[3,0].plot(range(1,len(model_history.history['val_bag_output_acc'])+1),model_history.history['val_bag_output_acc'])
    axs[3,0].set_title('Model bag_output_acc')
    axs[3,0].set_ylabel('bag_output_acc')
    axs[3,0].set_xlabel('Epoch')
    axs[3,0].set_xticks(np.arange(1,len(model_history.history['bag_output_acc'])+1),len(model_history.history['bag_output_acc'])/10)
    axs[3,0].legend(['train', 'val'], loc='best')

    axs[3,1].plot(range(1,len(model_history.history['footwear_output_acc'])+1),model_history.history['footwear_output_acc'])
    axs[3,1].plot(range(1,len(model_history.history['val_footwear_output_acc'])+1),model_history.history['val_footwear_output_acc'])
    axs[3,1].set_title('Model footwear_output_acc')
    axs[3,1].set_ylabel('footwear_output_acc')
    axs[3,1].set_xlabel('Epoch')
    axs[3,1].set_xticks(np.arange(1,len(model_history.history['footwear_output_acc'])+1),len(model_history.history['footwear_output_acc'])/10)
    axs[3,1].legend(['train', 'val'], loc='best')

    axs[4,0].plot(range(1,len(model_history.history['pose_output_acc'])+1),model_history.history['pose_output_acc'])
    axs[4,0].plot(range(1,len(model_history.history['val_pose_output_acc'])+1),model_history.history['val_pose_output_acc'])
    axs[4,0].set_title('Model pose_output_acc')
    axs[4,0].set_ylabel('pose_output_acc')
    axs[4,0].set_xlabel('Epoch')
    axs[4,0].set_xticks(np.arange(1,len(model_history.history['pose_output_acc'])+1),len(model_history.history['pose_output_acc'])/10)
    axs[4,0].legend(['train', 'val'], loc='best')

    axs[4,1].plot(range(1,len(model_history.history['emotion_output_acc'])+1),model_history.history['emotion_output_acc'])   
    axs[4,1].plot(range(1,len(model_history.history['val_emotion_output_acc'])+1),model_history.history['val_emotion_output_acc'])
    axs[4,1].set_title('Model emotion_output_acc')
    axs[4,1].set_ylabel('emotion_output_acc')
    axs[4,1].set_xlabel('Epoch')
    axs[4,1].set_xticks(np.arange(1,len(model_history.history['emotion_output_acc'])+1),len(model_history.history['emotion_output_acc'])/10)
    axs[4,1].legend(['train', 'val'], loc='best')


    # summarize history for loss
    axs[5,0].plot(range(1,len(model_history.history['gender_output_loss'])+1),model_history.history['gender_output_loss'])
    axs[5,0].plot(range(1,len(model_history.history['val_gender_output_loss'])+1),model_history.history['val_gender_output_loss'])
    axs[5,0].set_title('Model Loss')
    axs[5,0].set_ylabel('Loss')
    axs[5,0].set_xlabel('Epoch')
    axs[5,0].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[5,0].legend(['train', 'val'], loc='best')

    axs[5,1].plot(range(1,len(model_history.history['image_quality_output_loss'])+1),model_history.history['image_quality_output_loss'])
    axs[5,1].plot(range(1,len(model_history.history['val_image_quality_output_loss'])+1),model_history.history['val_image_quality_output_loss'])
    axs[5,1].set_title('Model Loss')
    axs[5,1].set_ylabel('Loss')
    axs[5,1].set_xlabel('Epoch')
    axs[5,1].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[5,1].legend(['train', 'val'], loc='best')

    axs[6,0].plot(range(1,len(model_history.history['age_output_loss'])+1),model_history.history['age_output_loss'])
    axs[6,0].plot(range(1,len(model_history.history['val_age_output_loss'])+1),model_history.history['val_age_output_loss'])
    axs[6,0].set_title('Model Loss')
    axs[6,0].set_ylabel('Loss')
    axs[6,0].set_xlabel('Epoch')
    axs[6,0].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[6,0].legend(['train', 'val'], loc='best')

    axs[6,1].plot(range(1,len(model_history.history['weight_output_loss'])+1),model_history.history['weight_output_loss'])
    axs[6,1].plot(range(1,len(model_history.history['val_weight_output_loss'])+1),model_history.history['val_weight_output_loss'])
    axs[6,1].set_title('Model Loss')
    axs[6,1].set_ylabel('Loss')
    axs[6,1].set_xlabel('Epoch')
    axs[6,1].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[6,1].legend(['train', 'val'], loc='best')

    axs[7,0].plot(range(1,len(model_history.history['bag_output_loss'])+1),model_history.history['bag_output_loss'])
    axs[7,0].plot(range(1,len(model_history.history['val_bag_output_loss'])+1),model_history.history['val_bag_output_loss'])
    axs[7,0].set_title('Model Loss')
    axs[7,0].set_ylabel('Loss')
    axs[7,0].set_xlabel('Epoch')
    axs[7,0].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[7,0].legend(['train', 'val'], loc='best')

    axs[7,1].plot(range(1,len(model_history.history['footwear_output_loss'])+1),model_history.history['footwear_output_loss'])
    axs[7,1].plot(range(1,len(model_history.history['val_footwear_output_loss'])+1),model_history.history['val_footwear_output_loss'])
    axs[7,1].set_title('Model Loss')
    axs[7,1].set_ylabel('Loss')
    axs[7,1].set_xlabel('Epoch')
    axs[7,1].set_xticks(np.arange(1,len(model_history.history['gender_output_loss'])+1),len(model_history.history['gender_output_loss'])/10)
    axs[7,1].legend(['train', 'val'], loc='best')

    
    plt.show()

In [4]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [5]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, aug_list=[], incl_orig=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        #self.augmentation = augmentation
        self.aug_list = aug_list
        self.incl_orig = incl_orig
        self.orig_len = int(np.floor(self.df.shape[0] / self.batch_size))

    def __len__(self):
        if self.incl_orig:
            delta = 1
        else:
            delta = 0
        return self.orig_len * (len(self.aug_list) + delta)

    def __getitem__(self, index):
        """fetch batched images and targets"""
        if not self.incl_orig :
            index += self.orig_len - 1

        if index > self.orig_len - 1:
            aug = self.aug_list[index // self.orig_len - 1]
            index %= self.orig_len
        else:
            aug = None

        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])        
        images = images.astype('float32')/255
        images -= np.mean(images, axis=0)

        if aug is not None:
            images = aug.flow(images, shuffle=False).next()

        #image /= np.std(image, axis=0)

        # images = np.array([io.imread(data_dir / p) for p in df.iloc[:, -1]])
        # mean = []
        # std = []
        # for i in range(images.shape[-1]):
        #     pixels = images[:, :, :, i].ravel()
        #     mean.append(np.mean(pixels))
        #     std.append(np.std(pixels))
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        #print(image.shape)
        #print(target)
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [7]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [8]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
58,resized/59.jpg,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
2106,resized/2107.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0
5206,resized/5207.jpg,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
1163,resized/1164.jpg,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
13534,resized/13536.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)#, augmentation=None)#ImageDataGenerator(horizontal_flip=True))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)


In [18]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu',
                 batch_normalization=True, conv_first=True):
    conv = Conv2D(num_filters, kernel_size=kernel_size, strides=strides,
                  padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [20]:
input_tensor=Input(shape=(224, 224, 3))

#num_filters = 16
#x = resnet_layer(inputs=input_tensor, num_filters=16, kernel_size=7, strides=2)
#x = MaxPooling2D(pool_size=(2,2))(x)
#x = resnet_layer(inputs=x)
## Instantiate the stack of residual units
#for stack in range(3):
#    for res_block in range(5):
#        strides = 1
#        if stack > 0 and res_block == 0:  # first layer but not first stack
#            strides = 2  # downsample
#        y = resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
#        y = resnet_layer(inputs=y, num_filters=num_filters, activation=None)
#        if stack > 0 and res_block == 0:  # first layer but not first stack
#            # linear projection residual shortcut connection to match
#            # changed dims
#            x = resnet_layer(inputs=x, num_filters=num_filters, kernel_size=1,
#                             strides=strides, activation=None, batch_normalization=False)
#        x = keras.layers.add([x, y])
#        x = Activation('relu')(x)
#    num_filters *= 2

x = Conv2D(16, 3, padding='same', kernel_regularizer=l2(1e-4), use_bias=False)(input_tensor)#3,224
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(32, 3, padding ='same', use_bias=False)(x)#5,224
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, padding ='same', use_bias=False)(x)#7,224
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = MaxPooling2D(pool_size=(2,2))(x)#8,110

x = Conv2D(16, 3, use_bias=False)(x)#12,108
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(32, 3, use_bias=False)(x)#16,106
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, use_bias=False)(x)#20,104
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = MaxPooling2D(pool_size=(2,2))(x)#22,52

x = Conv2D(16, 3, padding='same', use_bias=False)(x)#30,52
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(32, 3, padding ='same', use_bias=False)(x)#38,52
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, padding ='same', use_bias=False)(x)#46,52
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = MaxPooling2D(pool_size=(2,2))(x)#50,26

x = Conv2D(16, 3, use_bias=False)(x)#66,24
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(32, 3, use_bias=False)(x)#72,22
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, use_bias=False)(x)#88,20
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = MaxPooling2D(pool_size=(2,2))(x)#96,10

x = Conv2D(16, 3, padding='same', use_bias=False)(x)#128,10
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(32, 3, padding ='same', use_bias=False)(x)#160,10
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, padding ='same', use_bias=False)(x)#192,10
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(64, 3, padding ='same', use_bias=False)(x)#224,10
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Conv2D(8, 1, use_bias=False)(x)#224,10
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)


x = AveragePooling2D(pool_size=2)(x)
n = Flatten()(x)


#neck = backbone.output
#neck = Flatten(name="flatten")(neck)
#neck = Dense(512, activation="relu")(neck)


#def build_tower(in_layer):
#    neck = Dropout(0.2)(in_layer)
#    neck = Dense(128, activation="relu")(neck)
#    neck = Dropout(0.3)(in_layer)
#    neck = Dense(128, activation="relu")(neck)
#    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", n)
image_quality = build_head("image_quality", n)
age = build_head("age", n)
weight = build_head("weight", n)
bag = build_head("bag", n)
footwear = build_head("footwear", n)
emotion = build_head("emotion", n)
pose = build_head("pose", n)


model = Model(
    inputs=input_tensor, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 224, 224, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 224, 224, 16) 0         

In [21]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
path = os.path.join(save_dir, 'model_{epoch:03d}.hdf5')
print(path)

checkpoint = ModelCheckpoint(path,
                             save_best_only=True, verbose=1,
														 monitor='val_loss', mode='min')
														 
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, 
                               patience=3, min_lr=1e-6, verbose=1)
#lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_reducer]

# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}

opt = SGD(lr=0.01, momentum=0.85, nesterov=True)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"] 
              # loss_weights=loss_weights
             )

/content/saved_models/model_{epoch:03d}.hdf5




In [22]:
model_info= model.fit_generator(generator=train_gen, validation_data=valid_gen,
    use_multiprocessing=True, workers=6, 
    epochs=20, 
    callbacks=callbacks, 
    verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
360/360 [==============================] - 245s 681ms/step - loss: 7.8410 - gender_output_loss: 0.6631 - image_quality_output_loss: 0.9907 - age_output_loss: 1.4319 - weight_output_loss: 0.9944 - bag_output_loss: 0.9206 - footwear_output_loss: 0.9977 - pose_output_loss: 0.9327 - emotion_output_loss: 0.9086 - gender_output_acc: 0.5995 - image_quality_output_acc: 0.5497 - age_output_acc: 0.3990 - weight_output_acc: 0.6324 - bag_output_acc: 0.5624 - footwear_output_acc: 0.5128 - pose_output_acc: 0.6172 - emotion_output_acc: 0.7157 - val_loss: 7.8955 - val_gender_output_loss: 0.6473 - val_image_quality_output_loss: 0.9814 - val_age_output_loss: 1.4549 - val_weight_output_loss: 0.9833 - val_bag_output_loss: 0.9395 - val_footwear_output_loss: 0.9948 - val_pose_output_loss: 0.9290 - val_emotion_output_loss: 0.9640 - val_gender_output_acc: 0.6154 - val_image_quality_output_acc: 0.5559 - v

In [23]:
#plot_model_history(model_info)
scores = model.evaluate_generator(generator=valid_gen, verbose =1)
dict(zip(model.metrics_names, scores))

31/31 [==============================] - 9s 291ms/step


{'age_output_acc': 0.3805443548387097,
 'age_output_loss': 1.4050314080330633,
 'bag_output_acc': 0.6219758064516129,
 'bag_output_loss': 0.8536572898587873,
 'emotion_output_acc': 0.6844758064516129,
 'emotion_output_loss': 0.9187358925419469,
 'footwear_output_acc': 0.6290322580645161,
 'footwear_output_loss': 0.8272802675923994,
 'gender_output_acc': 0.8009072580645161,
 'gender_output_loss': 0.42315275149960674,
 'image_quality_output_acc': 0.5549395161290323,
 'image_quality_output_loss': 0.9625127450112374,
 'loss': 6.928777833138743,
 'pose_output_acc': 0.7560483870967742,
 'pose_output_loss': 0.5851286736226851,
 'weight_output_acc': 0.6471774193548387,
 'weight_output_loss': 0.9513695586112237}

In [0]:
# create train and validation data generators
#train_gen = PersonDataGenerator(train_df, batch_size=32, aug_list=[ImageDataGenerator(horizontal_flip=True)],
#        incl_orig=False)  # Whether to include original images)
#valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)


In [0]:
#model.load_weights("/content/saved_models/model_020.hdf5")
#opt = SGD(lr=0.001, momentum=0.95, nesterov=True)
#model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"]             )
#model_info= model.fit_generator(generator=train_gen, validation_data=valid_gen,
#    use_multiprocessing=True, workers=6, 
#    epochs=20, 
#    callbacks=callbacks, 
#    verbose=1)